In [1]:
!pip install pandas==1.3.4
!pip install transformers==4.12.5
!pip install datasets==1.15.1
!pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 11.5 MB 13.1 MB/s eta 0:00:01
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 3.1 MB 21.0 MB/s eta 0:00:01
     |████████████████████████████████| 101 kB 1.1 MB/s ta 0:00:01
     |████████████████████████████████| 3.3 MB 29.3 MB/s eta 0:00:01
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 290 kB 23.2 MB/s eta 0:00:01
     |████████████████████████████████| 29.4 MB 19.5 MB/s eta 0:00:01
     |████████████████████████████████| 212 kB 26.8 MB/s eta 0:00:01
     |████████████████████████████████| 140 kB 39.0 MB/s eta 0:00:01
     |████████████████████████████████| 131 kB 28.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 29.4 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 30.5 MB/s eta 0:00:01
     |█████████████████

In [2]:
import os
import pickle

import pandas as pd

import numpy as np
import torch

import transformers
from transformers import BertTokenizer

import datasets
from datasets import concatenate_datasets
from datasets import Dataset
from datasets import ClassLabel
from datasets import DatasetDict

In [3]:
DATA_FOLDER = '/notebooks/Notebooks/general_notebooks/datasets'
DATA_FILE = '/notebooks/MTL_tasks/data/pe_dataset_for_mtl_w_combined.csv'
# FEATURES_1_FILE = '/raid/home/jeremiec/Data/persuasive_essays/features'
# FEATURES_2_FILE = '/raid/home/jeremiec/Data/persuasive_essays/features_2'

In [4]:
dataset_df = pd.read_csv(DATA_FILE)

In [5]:
dataset_df

,Unnamed: 0,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,...,relative_position_in_para_char,is_in_intro,relative_position_in_para_token,is_in_conclusion,is_first_in_para,is_last_in_para,nr_preceeding_comps_in_para,nr_following_comps_in_para,structural_fts_as_text,structural_fts_as_text_combined
0,0,essay001,T1,MajorClaim 503 575,we should attach more importance to cooperatio...,MajorClaim,MajorClaim,[],NaN,Linked,...,0.8599,1,0.8795,0,1,1,0,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...
1,1,essay001,T2,MajorClaim 2154 2231,a more cooperative attitudes towards life is m...,MajorClaim,MajorClaim,[],NaN,Linked,...,0.6723,0,0.6571,1,1,1,0,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...
2,2,essay001,T3,Claim 591 714,"through cooperation, children can learn about ...",Claim,Claim,[],Support,Linked,...,0.0228,0,0.0286,0,1,0,0,3,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...
3,3,essay001,T4,Premise 716 851,What we acquired from team work is not only ho...,Premise,Premise,[],Support,NotLinked,...,0.2260,0,0.2000,0,0,0,1,2,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...
4,4,essay001,T5,Premise 853 1086,"During the process of cooperation, children ca...",Premise,Premise,[],Support,NotLinked,...,0.4488,0,0.4476,0,0,0,2,1,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5968,5968,essay402,T11,Premise 1275 1339,indirectly they will learn how to socialize ea...,Premise,Premise,[],Support,NotLinked,...,0.5994,0,0.6100,0,0,0,4,3,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...
5969,5969,essay402,T12,Premise 1341 1388,That will make children getting lots of friends,Premise,Premise,[],Support,NotLinked,...,0.7043,0,0.7300,0,0,0,5,2,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...
5970,5970,essay402,T13,Premise 1393 1436,they can contribute positively to community,Premise,Premise,[],Support,Linked,...,0.7870,0,0.8200,0,0,0,6,1,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...
5971,5971,essay402,T14,Premise 1448 1525,playing sport makes children getting healthy a...,Premise,Premise,[],Support,NotLinked,...,0.8744,0,0.8900,0,0,1,7,0,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...


In [6]:
dataset_df = dataset_df.drop(columns = dataset_df.columns[0])

In [7]:
dataset_df.columns

Index(['essay_nr', 'component_id', 'label_and_comp_idxs', 'text', 'label_x',
       'label_ComponentType', 'relation_SupportAttack', 'label_RelationType',
       'label_LinkedNotLinked', 'split_y', 'essay', 'argument_bound_1',
       'argument_bound_2', 'argument_id', 'sentence', 'paragraph', 'para_nr',
       'total_paras', 'token_count', 'token_count_covering_para',
       'tokens_count_covering_sentence', 'preceeding_tokens_in_sentence_count',
       'succeeding_tokens_in_sentence_count', 'token_ratio',
       'relative_position_in_para_char', 'is_in_intro',
       'relative_position_in_para_token', 'is_in_conclusion',
       'is_first_in_para', 'is_last_in_para', 'nr_preceeding_comps_in_para',
       'nr_following_comps_in_para', 'structural_fts_as_text',
       'structural_fts_as_text_combined'],
      dtype='object')

In [8]:
dataset_df = dataset_df.drop(columns=['label_x'])

In [9]:
dataset_df.columns

Index(['essay_nr', 'component_id', 'label_and_comp_idxs', 'text',
       'label_ComponentType', 'relation_SupportAttack', 'label_RelationType',
       'label_LinkedNotLinked', 'split_y', 'essay', 'argument_bound_1',
       'argument_bound_2', 'argument_id', 'sentence', 'paragraph', 'para_nr',
       'total_paras', 'token_count', 'token_count_covering_para',
       'tokens_count_covering_sentence', 'preceeding_tokens_in_sentence_count',
       'succeeding_tokens_in_sentence_count', 'token_ratio',
       'relative_position_in_para_char', 'is_in_intro',
       'relative_position_in_para_token', 'is_in_conclusion',
       'is_first_in_para', 'is_last_in_para', 'nr_preceeding_comps_in_para',
       'nr_following_comps_in_para', 'structural_fts_as_text',
       'structural_fts_as_text_combined'],
      dtype='object')

In [12]:
dataset_df['para_nr'].value_counts()

2    1939
3    1896
4    1224
1     465
5     428
6      20
7       1
Name: para_nr, dtype: int64

In [13]:
dataset_df['essay_nr'][1200]

'essay085'

In [14]:
dataset_df['para_nr'][1200]

3

In [16]:
2/7 

0.2857142857142857

## encoding

In [50]:
def get_comp_class_mc(x):
    
    label = x.label_ComponentType
    
    if label == 'MajorClaim':
        
        return 1
    
    elif label == 'Claim':
        
        return 0
    
    elif label == 'Premise':
        
        return 0

In [51]:
def get_comp_class_cl(x):
    
    label = x.label_ComponentType
    
    if label == 'Claim':
        
        return 1
    
    elif label == 'MajorClaim':
        
        return 0
    
    elif label == 'Premise':
        
        return 0

In [52]:
def get_comp_class_prem(x):
    
    label = x.label_ComponentType
    
    if label == 'Premise':
        
        return 1
    
    elif label == 'Claim':
        
        return 0
    
    elif label == 'MajorClaim':
        
        return 0

In [53]:
def get_linked_integer(x):
    
    label = x.label_LinkedNotLinked
    
    if label == 'Linked':
        
        return 1
    
    elif label == 'NotLinked':
        
        return 0

In [54]:
def get_support_integer(x):
    
    label = x.label_RelationType
    
    if label == 'Support':
        
        return 1
    
    else:
        
        return 0

In [55]:
def get_attack_integer(x):
    
    label = x.label_RelationType
    
    if label == 'Attack':
        
        return 1
    
    else:
        
        return 0

In [56]:
dataset_df['mc'] = dataset_df.apply(lambda x: get_comp_class_mc(x), axis=1)

In [57]:
dataset_df['cl'] = dataset_df.apply(lambda x: get_comp_class_cl(x), axis=1)

In [58]:
dataset_df['prem'] = dataset_df.apply(lambda x: get_comp_class_prem(x), axis=1)

In [59]:
dataset_df['link_present'] = dataset_df.apply(lambda x: get_linked_integer(x), axis=1)

In [60]:
dataset_df['support'] = dataset_df.apply(lambda x: get_support_integer(x), axis=1)

In [61]:
dataset_df['attack'] = dataset_df.apply(lambda x: get_attack_integer(x), axis=1)

In [62]:
dataset_df['mc'].value_counts()

0    5239
1     734
Name: mc, dtype: int64

In [63]:
dataset_df['cl'].value_counts()

0    4497
1    1476
Name: cl, dtype: int64

In [64]:
dataset_df['prem'].value_counts()

1    3763
0    2210
Name: prem, dtype: int64

In [65]:
dataset_df['link_present'].value_counts()

0    3566
1    2407
Name: link_present, dtype: int64

In [66]:
dataset_df['support'].value_counts()

1    4751
0    1222
Name: support, dtype: int64

In [67]:
dataset_df['attack'].value_counts()

0    5485
1     488
Name: attack, dtype: int64

In [68]:
train_df = dataset_df[dataset_df.split_y=='TRAIN'].reset_index(drop=True)
test_df = dataset_df[dataset_df.split_y=='TEST'].reset_index(drop=True)

# val_df = dataset_df[dataset_df.split=='VALIDATION'].reset_index(drop=True)
# train_dataset = train_val_datasets['train']
# val_dataset = train_val_datasets['test']

In [69]:
len(train_df), len(test_df)

(4713, 1260)

In [70]:
dataset_train = Dataset.from_pandas(train_df)
dataset_test = Dataset.from_pandas(test_df)

In [71]:
dataset_train['structural_fts_as_text'][0]

'Topic: Should students be taught to compete or to cooperate?, Sentence: From this point of view, I firmly believe that we should attach more importance to cooperation during primary education., Para Number: 1, First in Para: Yes, Last in Para: Yes, Is in Introduction: Yes, Is in Conclusion: No'

In [72]:
dataset_test

Dataset({
    features: ['essay_nr', 'component_id', 'label_and_comp_idxs', 'text', 'label_ComponentType', 'relation_SupportAttack', 'label_RelationType', 'label_LinkedNotLinked', 'split_y', 'essay', 'argument_bound_1', 'argument_bound_2', 'argument_id', 'sentence', 'paragraph', 'para_nr', 'total_paras', 'token_count', 'token_count_covering_para', 'tokens_count_covering_sentence', 'preceeding_tokens_in_sentence_count', 'succeeding_tokens_in_sentence_count', 'token_ratio', 'relative_position_in_para_char', 'is_in_intro', 'relative_position_in_para_token', 'is_in_conclusion', 'is_first_in_para', 'is_last_in_para', 'nr_preceeding_comps_in_para', 'nr_following_comps_in_para', 'structural_fts_as_text', 'structural_fts_as_text_combined', 'mc', 'cl', 'prem', 'link_present', 'support', 'attack'],
    num_rows: 1260
})

In [73]:
dataset = DatasetDict({"train": dataset_train, "test": dataset_test})

In [74]:
dataset

DatasetDict({
    train: Dataset({
        features: ['essay_nr', 'component_id', 'label_and_comp_idxs', 'text', 'label_ComponentType', 'relation_SupportAttack', 'label_RelationType', 'label_LinkedNotLinked', 'split_y', 'essay', 'argument_bound_1', 'argument_bound_2', 'argument_id', 'sentence', 'paragraph', 'para_nr', 'total_paras', 'token_count', 'token_count_covering_para', 'tokens_count_covering_sentence', 'preceeding_tokens_in_sentence_count', 'succeeding_tokens_in_sentence_count', 'token_ratio', 'relative_position_in_para_char', 'is_in_intro', 'relative_position_in_para_token', 'is_in_conclusion', 'is_first_in_para', 'is_last_in_para', 'nr_preceeding_comps_in_para', 'nr_following_comps_in_para', 'structural_fts_as_text', 'structural_fts_as_text_combined', 'mc', 'cl', 'prem', 'link_present', 'support', 'attack'],
        num_rows: 4713
    })
    test: Dataset({
        features: ['essay_nr', 'component_id', 'label_and_comp_idxs', 'text', 'label_ComponentType', 'relation_SupportAtt

In [75]:
dataset['test']

Dataset({
    features: ['essay_nr', 'component_id', 'label_and_comp_idxs', 'text', 'label_ComponentType', 'relation_SupportAttack', 'label_RelationType', 'label_LinkedNotLinked', 'split_y', 'essay', 'argument_bound_1', 'argument_bound_2', 'argument_id', 'sentence', 'paragraph', 'para_nr', 'total_paras', 'token_count', 'token_count_covering_para', 'tokens_count_covering_sentence', 'preceeding_tokens_in_sentence_count', 'succeeding_tokens_in_sentence_count', 'token_ratio', 'relative_position_in_para_char', 'is_in_intro', 'relative_position_in_para_token', 'is_in_conclusion', 'is_first_in_para', 'is_last_in_para', 'nr_preceeding_comps_in_para', 'nr_following_comps_in_para', 'structural_fts_as_text', 'structural_fts_as_text_combined', 'mc', 'cl', 'prem', 'link_present', 'support', 'attack'],
    num_rows: 1260
})

In [76]:
train_val_datasets = dataset['train'].train_test_split(train_size=0.8)
dataset_train = train_val_datasets['train']
dataset_val = train_val_datasets['test']

In [77]:
len(dataset_val)

943

In [78]:
len(dataset_train)

3770

In [79]:
len(dataset_test)

1260

In [80]:
dataset_train['structural_fts_as_text'][0]

'Topic: Gender Equality at university admission, Sentence: Therefore, universities follow the requirement of job providers and decide subject suitable for particular gender., Para Number: 3, First in Para: No, Last in Para: Yes, Is in Introduction: No, Is in Conclusion: No'

In [81]:
dataset_val['structural_fts_as_text'][0]

'Topic: Dormitory or apartment?, Sentence: From given the evidences, it seems to me that it becomes hard for students to choose between their living options since both living accommodation have its own benefits, Para Number: 4, First in Para: Yes, Last in Para: No, Is in Introduction: No, Is in Conclusion: Yes'

In [82]:
dataset_final = dataset = DatasetDict({"train": dataset_train, "test": dataset_test, "validation": dataset_val})

In [83]:
dataset_final

DatasetDict({
    train: Dataset({
        features: ['essay_nr', 'component_id', 'label_and_comp_idxs', 'text', 'label_ComponentType', 'relation_SupportAttack', 'label_RelationType', 'label_LinkedNotLinked', 'split_y', 'essay', 'argument_bound_1', 'argument_bound_2', 'argument_id', 'sentence', 'paragraph', 'para_nr', 'total_paras', 'token_count', 'token_count_covering_para', 'tokens_count_covering_sentence', 'preceeding_tokens_in_sentence_count', 'succeeding_tokens_in_sentence_count', 'token_ratio', 'relative_position_in_para_char', 'is_in_intro', 'relative_position_in_para_token', 'is_in_conclusion', 'is_first_in_para', 'is_last_in_para', 'nr_preceeding_comps_in_para', 'nr_following_comps_in_para', 'structural_fts_as_text', 'structural_fts_as_text_combined', 'mc', 'cl', 'prem', 'link_present', 'support', 'attack'],
        num_rows: 3770
    })
    test: Dataset({
        features: ['essay_nr', 'component_id', 'label_and_comp_idxs', 'text', 'label_ComponentType', 'relation_SupportAtt

In [84]:
torch.save(dataset_final, os.path.join("/notebooks/MTL_tasks/data", 'pe_dataset_for_MLC_w_combined.pt'))